In [145]:
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.linear_model import ElasticNet, Lasso, Ridge, BayesianRidge, LassoLarsIC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
import pandas as pd
import numpy as np
#Visualization packages
import matplotlib.pyplot as plt
import seaborn as sns

#NLP packages
from textblob import TextBlob

import warnings
warnings.filterwarnings("ignore")

In [208]:
train_data = pd.read_csv("train.csv", usecols=["class", "viewCount", "likeCount","dislikeCount","commentCount","title", "description", "user_comment_1", "user_comment_2", "user_comment_3", "user_comment_4", "user_comment_5", "user_comment_6", "user_comment_7", "user_comment_8", "user_comment_9", "user_comment_10"]) 
test_data = pd.read_csv("test_1.csv", usecols=["ID", "viewCount", "likeCount","dislikeCount","commentCount","title", "description", "user_comment_1", "user_comment_2", "user_comment_3", "user_comment_4", "user_comment_5", "user_comment_6", "user_comment_7", "user_comment_8", "user_comment_9", "user_comment_10"]) 
def Ratio(likes, dislikes):
    if likes == 0:
        return dislikes
    else:
        return dislikes/likes 

def num_caps(title):
    num = 0
    for i in title:
        if i.isupper():
           num += 1
    return num/len(title)
def num_exc(title):
    num = 0
    for i in title:
        if (i == '!') or (i == '(') or (i == ')') or (i == '[') or (i == ']') or (i == '{') or (i == '}') or (i == '#') or (i == '$') or (i == '*'):
            num+=1
    return num

def num_links(desc):
    spam = ['goo.gl', 'bit.ly', 'http', 'CHECK OUT', 'SUBSCRIBE', 'subscribe', 'Subscribe', 'link', 'Link', 'Twitter', 'Instagram', 'Twitch']
    num = 0
    for i in spam:
        if i in desc:
            num+=1
    return num

def num_digits(title):
    num = 0
    for i in title:
        if i.isdigit():
            num+=1
    return num
def calcPolarity(comment1, comment2, comment3, comment4, comment5, comment6, comment7, comment8, comment9, comment10):
    pol = 0
    sub = 0
    pol += TextBlob(comment1).sentiment.polarity
    pol += TextBlob(comment2).sentiment.polarity
    pol += TextBlob(comment3).sentiment.polarity
    pol += TextBlob(comment4).sentiment.polarity
    pol += TextBlob(comment5).sentiment.polarity
    pol += TextBlob(comment6).sentiment.polarity
    pol += TextBlob(comment7).sentiment.polarity
    pol += TextBlob(comment8).sentiment.polarity
    pol += TextBlob(comment9).sentiment.polarity
    pol += TextBlob(comment10).sentiment.polarity

    
    return (pol)/10
def calcSubjectivity(comment1, comment2, comment3, comment4, comment5, comment6, comment7, comment8, comment9, comment10):
    pol = 0
    sub = 0
    sub += TextBlob(comment1).sentiment.subjectivity
    sub += TextBlob(comment2).sentiment.subjectivity
    sub += TextBlob(comment3).sentiment.subjectivity
    sub += TextBlob(comment4).sentiment.subjectivity
    sub += TextBlob(comment5).sentiment.subjectivity
    sub += TextBlob(comment6).sentiment.subjectivity
    sub += TextBlob(comment7).sentiment.subjectivity
    sub += TextBlob(comment8).sentiment.subjectivity
    sub += TextBlob(comment9).sentiment.subjectivity
    sub += TextBlob(comment10).sentiment.subjectivity

    
    return sub/10
def indPolarity(comment):
    pol = 0
    pol+= TextBlob(comment).sentiment.polarity
    return pol

def indSubjectivity(comment):
    pol = 0
    pol+= TextBlob(comment).sentiment.subjectivity
    return pol
Y_train = train_data["class"]
X_test = test_data.drop("ID", axis = 1)
X_train = train_data.drop("class", axis=1)


X_train["Ratio"] = X_train.apply(lambda x: Ratio(x['likeCount'], x['dislikeCount']), axis = 1)
X_test["Ratio"] = X_test.apply(lambda x: Ratio(x['likeCount'], x['dislikeCount']), axis = 1)


X_train["dislikeCount"] = X_train["dislikeCount"].apply(lambda x: x)
X_test["dislikeCount"] = X_test["dislikeCount"].apply(lambda x: x)


X_train["polarity"] = X_train.apply(lambda x: calcPolarity(x["user_comment_1"], x["user_comment_2"], x["user_comment_3"], x["user_comment_4"], x["user_comment_5"], x["user_comment_6"], x["user_comment_7"], x["user_comment_8"], x["user_comment_9"], x["user_comment_10"]), axis = 1)
X_test["polarity"] = X_test.apply(lambda x: calcPolarity(x["user_comment_1"], x["user_comment_2"], x["user_comment_3"], x["user_comment_4"], x["user_comment_5"], x["user_comment_6"], x["user_comment_7"], x["user_comment_8"], x["user_comment_9"], x["user_comment_10"]), axis = 1)

X_train["subjectivity"] = X_train.apply(lambda x: calcSubjectivity(x["user_comment_1"], x["user_comment_2"], x["user_comment_3"], x["user_comment_4"], x["user_comment_5"], x["user_comment_6"], x["user_comment_7"], x["user_comment_8"], x["user_comment_9"], x["user_comment_10"]), axis = 1)
X_test["subjectivity"] = X_test.apply(lambda x: calcSubjectivity(x["user_comment_1"], x["user_comment_2"], x["user_comment_3"], x["user_comment_4"], x["user_comment_5"], x["user_comment_6"], x["user_comment_7"], x["user_comment_8"], x["user_comment_9"], x["user_comment_10"]), axis = 1)

X_train["user_comment_1"] = X_train["user_comment_1"].apply(lambda x: indPolarity(x))
X_test["user_comment_1"] = X_test["user_comment_1"].apply(lambda x: indPolarity(x))
X_train["user_comment_2"] = X_train["user_comment_2"].apply(lambda x: indPolarity(x))
X_test["user_comment_2"] = X_test["user_comment_2"].apply(lambda x: indPolarity(x))
X_train["user_comment_3"] = X_train["user_comment_3"].apply(lambda x: indPolarity(x))
X_test["user_comment_3"] = X_test["user_comment_3"].apply(lambda x: indPolarity(x))
X_train["user_comment_4"] = X_train["user_comment_4"].apply(lambda x: indPolarity(x))
X_test["user_comment_4"] = X_test["user_comment_4"].apply(lambda x: indPolarity(x))
X_train["user_comment_5"] = X_train["user_comment_5"].apply(lambda x: indPolarity(x))
X_test["user_comment_5"] = X_test["user_comment_5"].apply(lambda x: indPolarity(x))
X_train["user_comment_6"] = X_train["user_comment_6"].apply(lambda x: indPolarity(x))
X_test["user_comment_6"] = X_test["user_comment_6"].apply(lambda x: indPolarity(x))
X_train["user_comment_7"] = X_train["user_comment_7"].apply(lambda x: indPolarity(x))
X_test["user_comment_7"] = X_test["user_comment_7"].apply(lambda x: indPolarity(x))
X_train["user_comment_8"] = X_train["user_comment_8"].apply(lambda x: indPolarity(x))
X_test["user_comment_8"] = X_test["user_comment_8"].apply(lambda x: indPolarity(x))
X_train["user_comment_9"] = X_train["user_comment_9"].apply(lambda x: indPolarity(x))
X_test["user_comment_9"] = X_test["user_comment_9"].apply(lambda x: indPolarity(x))
X_train["user_comment_10"] = X_train["user_comment_10"].apply(lambda x: indPolarity(x))
X_test["user_comment_10"] = X_test["user_comment_10"].apply(lambda x: indPolarity(x))

X_train["num_caps"] = X_train.apply(lambda x: num_caps(x["title"]), axis = 1)
X_test["num_caps"] = X_test.apply(lambda x: num_caps(x["title"]), axis = 1)

X_train["num_exc"] = X_train.apply(lambda x: num_exc(x["title"]), axis = 1)
X_test["num_exc"] = X_test.apply(lambda x: num_exc(x["title"]), axis = 1)

X_train["num_digits"] = X_train.apply(lambda x: num_exc(x["title"]), axis = 1)
X_test["num_digits"] = X_test.apply(lambda x: num_exc(x["title"]), axis = 1)

X_train["num_links_desc"] = X_train.apply(lambda x: num_links(x["description"]), axis = 1)
X_test["num_links_desc"] = X_test.apply(lambda x: num_links(x["description"]), axis = 1)

X_train["num_caps_desc"] = X_train.apply(lambda x: num_caps(x["description"]), axis = 1)
X_test["num_caps_desc"] = X_test.apply(lambda x: num_caps(x["description"]), axis = 1)


X_train["pol_title"] = X_train["title"].apply(lambda x: indPolarity(x))
X_test["pol_title"] = X_test["title"].apply(lambda x: indPolarity(x))

X_train["subj_title"] = X_train["description"].apply(lambda x: indSubjectivity(x))
X_test["subj_title"] = X_test["description"].apply(lambda x: indSubjectivity(x))

X_train["title"] = X_train["title"].apply(lambda x: len(x))
X_test["title"] = X_test["title"].apply(lambda x: len(x))

X_train


X_train["pol_desc"] = X_train["description"].apply(lambda x: indPolarity(x))
X_test["pol_desc"] = X_test["description"].apply(lambda x: indPolarity(x))

X_train["subj_desc"] = X_train["description"].apply(lambda x: indSubjectivity(x))
X_test["subj_desc"] = X_test["description"].apply(lambda x: indSubjectivity(x))



X_train["description"] = X_train["description"].apply(lambda x: len(x))
X_test["description"] = X_test["description"].apply(lambda x: len(x))


X_train["Ratio"] = X_train["Ratio"].replace(np.nan, 0)
X_test["Ratio"] = X_test["Ratio"].replace(np.nan, 0)


X_train = X_train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
X_test = X_test.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
X_train.head()

,title,description,viewCount,likeCount,dislikeCount,commentCount,user_comment_1,user_comment_2,user_comment_3,user_comment_4,...,subjectivity,num_caps,num_exc,num_digits,num_links_desc,num_caps_desc,pol_title,subj_title,pol_desc,subj_desc
0,0.257732,0.241531,0.050176,0.105793,0.006975,0.050409,0.437500,0.50,0.500000,0.500000,...,0.153803,0.678571,0.166667,0.166667,0.333333,0.156867,0.50000,0.551515,0.508902,0.551515
1,0.505155,0.134295,0.000018,0.000004,0.000101,0.000010,0.658333,0.50,0.568182,0.583333,...,0.524499,0.134615,0.000000,0.000000,0.333333,0.200441,0.75000,0.716667,0.741667,0.716667
2,0.391753,0.114452,0.004843,0.004100,0.000611,0.003195,0.500000,0.50,0.650000,0.528125,...,0.318603,0.756098,0.500000,0.500000,0.666667,0.526203,0.71875,0.471970,0.580800,0.471970
3,0.453608,0.352175,0.000303,0.000234,0.000030,0.000121,0.503125,0.80,0.500000,0.800000,...,0.599169,0.106383,0.166667,0.166667,0.333333,0.141544,0.50000,0.472381,0.485119,0.472381
4,0.628866,0.177591,0.000980,0.000147,0.000119,0.000394,1.000000,0.56,0.580000,0.508333,...,0.472290,0.109375,0.000000,0.000000,0.333333,0.124861,0.52500,0.399074,0.693519,0.399074


In [209]:
model = RandomForestClassifier(n_estimators = 646)
model.fit(X_train, Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=646,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [210]:
X_test.head()

,title,description,viewCount,likeCount,dislikeCount,commentCount,user_comment_1,user_comment_2,user_comment_3,user_comment_4,...,subjectivity,num_caps,num_exc,num_digits,num_links_desc,num_caps_desc,pol_title,subj_title,pol_desc,subj_desc
0,1.000000,0.244588,0.003692,0.002744,0.003125,0.001486,0.60000,1.000000,0.458333,0.5000,...,0.476793,0.195000,0.0,0.0,0.666667,0.054147,0.500000,0.577646,0.548214,0.577646
1,0.191489,0.023730,0.031558,0.026252,0.009788,0.002673,0.65625,0.562500,0.400000,0.7500,...,0.488593,0.180556,0.0,0.0,0.000000,0.159338,0.500000,0.688889,0.505556,0.688889
2,0.436170,0.131557,0.000480,0.000301,0.000653,0.000136,0.51750,0.587500,0.800000,0.4125,...,0.582278,0.138298,0.0,0.0,1.000000,0.101075,0.568182,0.580000,0.561818,0.580000
3,0.276596,0.073064,0.042589,0.045089,0.015197,0.021510,0.50000,0.500000,0.750000,0.5000,...,0.392405,0.203125,0.6,0.6,1.000000,0.259743,0.500000,0.500000,0.750000,0.500000
4,0.340426,0.052456,0.132955,0.058901,0.025551,0.047855,0.60000,0.678571,0.500000,0.2500,...,0.437010,0.171053,0.2,0.2,0.666667,0.254401,0.000000,0.500000,0.750000,0.500000


In [211]:
Y_pred = model.predict(X_test)

test_data["class"] = Y_pred
test_data["class"] = test_data["class"].map(lambda x: "True" if x==1 else "False")
result = test_data[["ID","class"]]
result.to_csv("baseline2_result.csv", index=False)
result.head()

    
    

,ID,class
0,oRB8lJynqBA,True
1,of-UPoEnw_w,False
2,d28cz00HHto,False
3,Tftg_LnwTW0,False
4,qR0mkm65Whk,True


In [103]:
TextBlob("The movie is most crucial to the part that clickbait").sentiment


Sentiment(polarity=0.25, subjectivity=0.75)